In [1]:
import pinecone
from tqdm import tqdm
from dotenv import load_dotenv
import os

# Load environment variables from .env file
#overide = true just forces a reload on the .env file in case api key changes
load_dotenv(override=True)

# Access the API key
pinecone_api_key = os.getenv("PINECONE_API_KEY")
huggingface_api_token = os.getenv("HUGGING_FACE_API_TOKEN")


C:\Users\Brady\.vscode\omnistudy-AI_Testing\venv\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
#Initialize the pinecone index
import pinecone      

pinecone.init(      
	api_key=pinecone_api_key,      
	environment='gcp-starter'      
)      
index = pinecone.Index('haystack')

#load a pinecone document store object with the index defined previously
from haystack.document_stores import PineconeDocumentStore

document_store = PineconeDocumentStore(
    api_key=pinecone_api_key,
    pinecone_index=index,
    similarity="cosine",
    embedding_dim=768
)

In [3]:
#Initialize retriever model
from haystack.nodes import EmbeddingRetriever
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
    model_format="sentence_transformers"
)
from haystack.pipelines import DocumentSearchPipeline

search_pipe = DocumentSearchPipeline(retriever=retriever)

C:\Users\Brady\.vscode\omnistudy-AI_Testing\venv\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [41]:
query = "What is big O notation?"

In [42]:
docs = search_pipe.run(query=query, params={"Retriever": {"top_k": 2}})['documents']
documents = []
for doc in docs:
    documents.append(doc.content)




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
print("supporting documents provided")
for text in documents:
    print(text)

supporting documents provided
For example, the time to paint a fence that's w meters
wide and h meters high could be described as O ( wh). If you needed p layers of paint, then you could say
that the time is O ( whp).
Big 0, Big Theta, and Big Omega
If you've never covered big O in an academic setting, you can probably skip this subsection. It might
confuse you more than it helps. This "FYI" is mostly here to clear up ambiguity in wording for people who
have learned big O before, so that they don't say, "But I thought big O meant..:'
Academics use big 0, big 0 (theta), and big O (omega) to describe runtimes.
O (big 0): In academia, big O describes an upper bound on the time. An algorithm that prints all the
values in an array could be described as O(N), but it could also be described as O(N2), O(N3), or 0( 2N)
(or many other big O times). The algorithm is at least as fast as each of these; therefore they are upper
bounds on the runtime. This is similar to a less-than-or-equal-to relati

In [43]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "vblagoje/bart_lfqa"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)


conditioned_doc = "<P> " + " <P> ".join([d for d in documents])
query_and_docs = "question: {} context: {}".format(query, conditioned_doc)

model_input = tokenizer(query_and_docs, truncation=True, padding=True, return_tensors="pt")

generated_answers_encoded = model.generate(input_ids=model_input["input_ids"].to(device),
                                           attention_mask=model_input["attention_mask"].to(device),
                                           min_length=64,
                                           max_length=256,
                                           do_sample=False, 
                                           early_stopping=True,
                                           num_beams=8,
                                           temperature=1.0,
                                           top_k=None,
                                           top_p=None,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
tokenizer.batch_decode(generated_answers_encoded, skip_special_tokens=True,clean_up_tokenization_spaces=True)



["Big O is an upper bound on the time it takes to do something. For example, the time to paint a fence that's w meters wide and h meters high could be described as O (wh). If you needed p layers of paint, then you could say that the time is O ( whp). Theta and Omega are upper bounds on the runtimes of algorithms."]

In [15]:
from haystack import Pipeline
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

query = "How to prepare for coding interviews?"

prompt_node = PromptNode()
output = prompt_node.prompt(prompt_template="deepset/question-answering",
                   query=query,
                   documents=search_pipe.run(query=query,params={"Retriever": {"top_k": 1}})['documents'])

print(output)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['test your code-on paper']


In [9]:
print(output[1])

IndexError: list index out of range

In [5]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser
from haystack import Document

pipe.add_node(component=node,name="prompt_node",inputs=["Query"])

In [6]:
query = "What kind of Big O notation is used during a coding interview?"
res=pipe.run(query=query,documents=search_pipe.run(query=query,params={"Retriever": {"top_k": 2}})['documents'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Exception: Exception while running node 'prompt_node': HuggingFace Inference returned an error.
Status code: 403
Response body: {"error":"The model google/flan-t5-xl is too large to be loaded automatically (11GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints)."}
Enable debug logging to see the data that was passed when the pipeline failed.

In [ ]:
print(res['answers'])